In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [13]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [14]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [15]:
# trips_generator_period = 10 else simulation is v. slow
simulator = Simulator(trips_generator_period=10)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
sampled_X = []  # keep track of points we sampled for debugging
sampled_Y = []

def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray """
    result = []
    print(X)
    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = int(edgeLength),
            numberOfLanes = int(numLanes),
            accel         = accel
        )
        result.append(s['timeLoss'])
        sampled_X.append([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel])
        sampled_Y.append(s['timeLoss'])
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. I think that emukit handles categorical/discrete inputs using one-hot encodings.

#### Model Inputs

In [6]:
gridSize = ContinuousParameter('gridSize', min_value=3, max_value=20)
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=1, max_value=25)
maxSpeed = ContinuousParameter('maxSpeed', min_value=1, max_value=25)
edgeLength = ContinuousParameter('edgeLength', min_value=2, max_value=200)
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
accel = ContinuousParameter('accel', 1., 6.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numberOfLanes, accel])

#### Initialize Model / Emulator (GP)

In [7]:
init_X, init_Y = pickle.load(open('init_and_test_points/init_points_6_param.pkl', 'rb'))

In [8]:
init_Y = init_Y[:,:,0]  # get timeLoss only

In [9]:
# emulator = pickle.load(open('experimental_design_timeLoss_50_iter_batch_size_3.pkl',"rb"))
emulator = GPy.models.GPRegression(init_X[::2], init_Y[::2])
emukit_model = GPyModelWrapper(emulator)
emulator

GP_regression.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,1.0,+ve,
Gaussian_noise.variance,1.0,+ve,


### Optimization

#### Acquisition Function

In [10]:
model_variance = ModelVariance(model=emukit_model)

#### Experimental Design

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = model_variance,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 100)

Optimization restart 1/1, f = 723.0692657614825
[[  3.  25.  25. 200.   3.   1.]]
 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 729.0056592151842
[[  3.          25.           1.         101.49673835   1.
    1.        ]]
 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 735.1646084878499
[[20.          1.         25.         95.53046185  1.          1.        ]]
 Retrying in 1 seconds


KeyboardInterrupt: 

### Evaluate

#### Test on 25 point test set

In [25]:
test_X, test_Y = pickle.load(open('init_and_test_points/test_points_6_param.pkl', 'rb'))
test_Y = test_Y[:,:,0]
pred_Y, pred_Y_variance = emukit_model.predict(test_X)

In [26]:
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  115.94522232155397


In [27]:
pred_Y, test_Y

(array([[6.80067495e-09],
        [2.95017931e-05],
        [5.79961086e-12],
        [2.67052412e-11],
        [1.52860367e-17],
        [2.35930943e-13],
        [2.49659277e-07],
        [3.51761479e-08],
        [5.53262510e-08],
        [1.20760352e-01],
        [1.83935078e-12],
        [2.32251183e-05],
        [1.07224765e-07],
        [2.53862409e-11],
        [3.66510598e-16],
        [5.52814614e-05],
        [1.97711394e-09],
        [2.14373388e-15],
        [3.16058668e-16],
        [3.51650875e-17],
        [1.96723848e-12],
        [5.04609241e-06],
        [1.00661440e-06],
        [6.94336006e-17],
        [9.15407720e-04]]),
 array([[ 18.42],
        [ 11.64],
        [ 36.32],
        [ 98.25],
        [ 20.55],
        [ 22.81],
        [ 29.6 ],
        [  5.23],
        [ 12.61],
        [ 50.47],
        [ 13.7 ],
        [ 13.91],
        [372.67],
        [  6.95],
        [ 71.88],
        [175.18],
        [ 22.55],
        [  3.5 ],
        [ 26.76],
      

### Save Model

In [20]:
with open('timeLoss_model_variance_100_iter_batch_size_1.pkl', "wb") as f:
     pickle.dump(emulator, f)

### Analysis

As we can see, optimizing for model_variance causes emukit to alternate between extreme choices of \[gridSize, edgeMaxSpeed, maxSpeed, numberOfLanes, accel\] whilst only varying edgeLength sensibly. This means that we don't explore the parameter space comprehensively - which is undesirable.